In [1]:
#clean environment
from IPython import get_ipython
get_ipython().magic('reset -f')
from sqlalchemy import create_engine
import pandas as pd
import pyodbc
import json
import requests
import numpy as np
from datetime import datetime

'''
ask token
'''    
url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
payload = {
    "grant_type": "password",
    "client_id": "351fb11916fc85504ba5f6ec3b96988b",
    "client_secret": "8f9355878504005ba782ca049d013d4e",
    "redirect_uri": "https://crm-p10.xiaoshouyi.com",
    "username": "11020964@twkd.com",
    "password": "Kd0627ubGHagpQ"
}
response = requests.post(url, data=payload)
content = response.json()
ac_token = content["access_token"]


current_month = datetime.now().month

# =============================================================================
# '''
# select from contact
# '''
# url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
# headers = {
#     "Authorization": f"Bearer {ac_token}",
#     "Content-Type":"application/x-www-form-urlencoded"
#     # Replace with your actual access token
# }
# queryLocator = ''
# contact_df = pd.DataFrame()
#  
# while True:
#     data = {
#         "xoql": '''SELECT id, customItem182__c as ContactID, accountCode__c as 公司代號, accountId.accountName as 公司全名, 
#                    contactCode__c as 聯絡人代號, contactName as 聯絡人名稱, mobile as 聯絡人手機, phone as 公司電話,
#                    customItem192__c as 聯絡人生日, dimDepart.departName as 客戶資料區域, 
#                    ownerId.name as 聯絡人所有人 FROM contact 
#                           ''',
#         "batchCount": 2000,
#         "queryLocator": queryLocator
#     }
#     response = requests.post(url_2, headers=headers, data=data)
#     crm = response.json()
#     data = pd.DataFrame(crm["data"]["records"])
#     contact_df = pd.concat([contact_df, data], ignore_index=True, sort=False)
#     
#     if not crm['queryLocator']:
#         break
#     queryLocator = crm['queryLocator']
# =============================================================================

'''
select from related_contact
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
contact_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''SELECT id, customItem8__c as 公司代號, customItem1__c.accountName as 公司全名, name as 關係聯絡人代號,
                   contactCode__c__c as 聯絡人代號, customItem2__c.contactName as 聯絡人名稱, contactPhone__c__c as 聯絡人手機, customItem2__c.phone as 公司電話,
                   customItem2__c.customItem192__c as 聯絡人生日, dimDepart.departName as 客戶資料區域, 
                   ownerId.name as 聯絡人所有人, customItem95__c as 職務類別, customItem24__c as 關係狀態  FROM customEntity22__c
                   where (customItem95__c = '005 設計師' or customItem95__c = '004 設計總監/設計主管類')
                          ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    contact_df = pd.concat([contact_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

birth_45 = contact_df[~contact_df['聯絡人生日'].isin(['', 'MM/DD'])]


# =============================================================================
# '''
# select from related_contact
# '''
# url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
# headers = {
#     "Authorization": f"Bearer {ac_token}",
#     "Content-Type":"application/x-www-form-urlencoded"
#     # Replace with your actual access token
# }
# queryLocator = ''
# related_df = pd.DataFrame()
#  
# while True:
#     data = {
#         "xoql": '''select id, contactCode__c__c as 聯絡人代號, customItem95__c as 職務類別, customItem24__c, customItem74__c as LineID
#                    from customEntity22__c
#                    where (customItem95__c = '005 設計師' or customItem95__c = '004 設計總監' and (customItem24__c = '在職（主要公司）'))
#                    ''',
#         "batchCount": 2000,
#         "queryLocator": queryLocator
#     }
#     response = requests.post(url_2, headers=headers, data=data)
#     crm = response.json()
#     data = pd.DataFrame(crm["data"]["records"])
#     related_df = pd.concat([related_df, data], ignore_index=True, sort=False)
#     
#     if not crm['queryLocator']:
#         break
#     queryLocator = crm['queryLocator']
#     
# birth_45 = pd.merge(birth, related_df, on = '聯絡人代號', how = 'left')
# birth_45 = birth_45[~birth_45['id_y'].isna()] #########################################在聯絡人不在關係聯絡人
#     
# =============================================================================
    
def format_birthday(birthday):
    parts = birthday.split('/')
    if len(parts) == 2:
        return '{:0>2}/{:0>2}'.format(parts[0], parts[1])
    else:
        return birthday

# Apply the formatting function to the '聯絡人生日' column
birth_45['聯絡人生日'] = birth_45['聯絡人生日'].apply(format_birthday)
birth_45['聯絡人生日'] = birth_45['聯絡人生日'].apply(lambda x: x[5:] if len(x) == 10 else x)
birth_45['length'] = birth_45['聯絡人生日'].str.len()

next_month = (current_month % 12) + 1

# Function to check if the birthday is in the next month
def is_next_month_birthday(date_str):
    try:
        # Parse the date string
        birthday = datetime.strptime(date_str.replace('/', '-'), '%m-%d')
        
        # Check if the birthday month is the next month
        return birthday.month == next_month
    except ValueError:
        return False

# Apply the function to filter rows
next_month_birthdays = birth_45[birth_45['聯絡人生日'].apply(is_next_month_birthday)]
df_no_duplicates = next_month_birthdays.drop_duplicates(subset=['聯絡人名稱', '聯絡人手機'])
df_no_duplicates['是否發送生日禮'] = ''
df_no_duplicates['實際發送人員'] = ''
df_no_duplicates['備註'] = ''
df_no_duplicates['關係狀態'] = df_no_duplicates['關係狀態'].str.get(0)
df_no_duplicates = df_no_duplicates[df_no_duplicates['關係狀態'].str.contains('主要')]
columns_to_drop = ['職務類別', '關係狀態', 'length']
df_no_duplicates = df_no_duplicates.drop(columns=columns_to_drop)
df_no_duplicates = df_no_duplicates[df_no_duplicates['聯絡人手機']!='']
df_no_duplicates = df_no_duplicates.drop_duplicates(subset=['聯絡人代號'], keep='first')
df_tw = df_no_duplicates[df_no_duplicates['客戶資料區域'].str.contains('TW')]
df_ov = df_no_duplicates[~df_no_duplicates['聯絡人代號'].isin(df_tw['聯絡人代號'])]
df_tw.to_excel(f'{next_month}月台灣設計師生日禮清單.xlsx', index = False)
df_ov.to_excel(f'{next_month}月海外設計師生日禮清單.xlsx', index = False)

# =============================================================================
# '''
# connect to CRM
# '''
# server = '192.168.1.112'
# #database = 'TEMP_GDCRM'
# database = 'GDCRM'
# username = 'sa'
# password = 'mis528716#'
# crm_con = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 
# cursor = crm_con.cursor()
# engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password))
# crm_birth_str = '''SELECT 
# 	           ContactID,
#                Birthday
#                from Contact con join Company c
#                on con.OrigID = c.CompanyID
#                join u_DataRegion u on c.DataRegionID = u.DataRegionID
#                where (u.DataRegionGroupID like'TW-%') and (Birthday is not null and Birthday <>'')
#             '''
# crm_birth_df = pd.read_sql_query(crm_birth_str, crm_con)
# 
# crm_birth = pd.merge(crm_birth_df, contact_df, on = 'ContactID', how = 'left')
# =============================================================================


# =============================================================================
# '''
# select from related_contact
# '''
# url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
# headers = {
#     "Authorization": f"Bearer {ac_token}",
#     "Content-Type":"application/x-www-form-urlencoded"
#     # Replace with your actual access token
# }
# queryLocator = ''
# related_df = pd.DataFrame()
#  
# while True:
#     data = {
#         "xoql": "select id, customItem98__c as ContactID from customEntity22__c where customItem98__c is not null",
#         "batchCount": 2000,
#         "queryLocator": queryLocator
#     }
#     response = requests.post(url_2, headers=headers, data=data)
#     crm = response.json()
#     data = pd.DataFrame(crm["data"]["records"])
#     related_df = pd.concat([related_df, data], ignore_index=True, sort=False)
#     
#     if not crm['queryLocator']:
#         break
#     queryLocator = crm['queryLocator']
#  
# print(related_df)
# 
# crm_related_contact = pd.merge(crm_birth_df, related_df, on = 'ContactID', how = 'left')
# crm_related_nan = crm_related_contact[crm_related_contact['id'].isna()]
# crm_related_nan.to_excel('na.xlsx', index = False)
# =============================================================================


C:\Users\11020964\AppData\Local\Temp\ipykernel_24016\2027095579.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -f')
C:\Users\11020964\AppData\Local\Temp\ipykernel_24016\2027095579.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  birth_45['聯絡人生日'] = birth_45['聯絡人生日'].apply(format_birthday)
C:\Users\11020964\AppData\Local\Temp\ipykernel_24016\2027095579.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning